<a href="https://colab.research.google.com/github/Rumeysakeskin/Traffic-Signs-Detection-with-YOLO/blob/main/traffic_sign_detection_YOLOv3_Darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALL DARKNET**

Darknet: It is an open-source neural network framework. We will use it to train the pre-trained weights later.

In [ ]:
# Clone the darknet
!git clone https://github.com/AlexeyAB/darknet
%cd darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.35 MiB | 13.97 MiB/s, done.
Resolving deltas: 100% (10670/10670), done.
/content/darknet


In [ ]:
# change makefile to have GPU and OPENCV enable
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make
!./darknet

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘bu

Now we need pre-trained weights, you can use darknet53.conv.74. This weight is going to be retrained on given data.

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74
!ls

--2023-12-01 21:44:23--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  46.1MB/s    in 3.8s    

2023-12-01 21:44:27 (40.3 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]

3rdparty	darknet			docker-compose.yml     LICENSE	      results
backup		darknet53.conv.74	Dockerfile.cpu	       Makefile       scripts
build		DarknetConfig.cmake.in	Dockerfile.gpu	       net_cam_v3.sh  src
build.ps1	darknet_images.py	image_yolov3.sh        net_cam_v4.sh  vcpkg.json
cfg		darknet.py		image_yolov4.sh        obj	      vcpkg.json.opencv23
cmake		darknet_video.py	include		       package.xml    video_yolov3.sh
CMakeLists.txt	data			json_mjpeg_streams.sh  README.md      video_yolov4.sh

**CHECK DARKNET INSTALLATION**

In [ ]:
!./darknet detect cfg/yolov3.cfg weights/yolov3.weights data/test-image.jpg

 CUDA-version: 11080 (12000), cuDNN: 8.9.6, GPU count: 1  
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  

**DATA PREPROCESSING**

In [ ]:
# Install and unzip dataset
import zipfile, urllib.request, shutil
url = "https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip"
file_name = 'FullIJCNN2013.zip'

with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
    with zipfile.ZipFile(file_name) as zf:
        zf.extractall()

In [ ]:
import os
import pandas as pd
import cv2


*   LIST OF CATEGORIES




In [ ]:
# Defining lists for categories according to the classes ID's
# Prohibitory category:
# circular Traffic Signs with white background and red border line
p = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]

# Danger category:
# triangular Traffic Signs with white background and red border line
d = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

# Mandatory category:
# circular Traffic Signs with blue background
m = [33, 34, 35, 36, 37, 38, 39, 40]

# Other category:
o = [6, 12, 13, 14, 17, 32, 41, 42]


*   LOADING ORIGINAL ANNOTATIONS



In [ ]:
ann = pd.read_csv("FullIJCNN2013" + '/' + 'gt.txt',
                  names=['ImageID', 'XMin', 'YMin', 'XMax', 'YMax', 'ClassID'],
                  sep=';')
ann.head()

,ImageID,XMin,YMin,XMax,YMax,ClassID
0,00000.ppm,774,411,815,446,11
1,00001.ppm,983,388,1024,432,40
2,00001.ppm,386,494,442,552,38
3,00001.ppm,973,335,1031,390,13
4,00002.ppm,892,476,1006,592,39



*   CALCULATION OF BOUNDING BOX'S PARAMETERS





In [ ]:
# Adding new empty columns to dataFrame to save numbers for YOLO format
ann['CategoryID'] = ''
ann['center x'] = ''
ann['center y'] = ''
ann['width'] = ''
ann['height'] = ''

# Getting category's ID according to the class's ID
ann.loc[ann['ClassID'].isin(p), 'CategoryID'] = 0
ann.loc[ann['ClassID'].isin(d), 'CategoryID'] = 1
ann.loc[ann['ClassID'].isin(m), 'CategoryID'] = 2
ann.loc[ann['ClassID'].isin(o), 'CategoryID'] = 3

# Calculating bounding box's center in x and y for all rows
ann['center x'] = (ann['XMax'] + ann['XMin']) / 2
ann['center y'] = (ann['YMax'] + ann['YMin']) / 2

# Calculating bounding box's width and height for all rows
ann['width'] = ann['XMax'] - ann['XMin']
ann['height'] = ann['YMax'] - ann['YMin']

In [ ]:
# By using copy() we create separate dataFrame and initial dataFrame will not be changed.
r = ann.loc[:, ['ImageID',
                'CategoryID',
                'center x',
                'center y',
                'width',
                'height']].copy()
r.head()

,ImageID,CategoryID,center x,center y,width,height
0,00000.ppm,1,794.5,428.5,41,35
1,00001.ppm,2,1003.5,410.0,41,44
2,00001.ppm,2,414.0,523.0,56,58
3,00001.ppm,3,1002.0,362.5,58,55
4,00002.ppm,2,949.0,534.0,114,116



*   NORMALIZING BOUNDING BOX'S PARAMETERS
*   SAVING ANNOTATONS IN TXT FILES
*   COVERTING IMAGES FROM PPM TO JPG
















In [ ]:

path_dir = "/content/darknet/FullIJCNN2013/"
path = os.chdir(path_dir)
path = os.listdir(path_dir)

for file in path:

    # Checking if filename ends with '.ppm'
    if file.endswith('.ppm'):
        # Reading image and getting its real width and height
        image_ppm = cv2.imread(file)

        # Slicing from tuple only first two elements
        h, w = image_ppm.shape[:2]
        # Slicing only name of the file without extension
        image_name = file[:-4]

        sub_r = r.loc[r['ImageID'] == file].copy()

        # Normalizing calculated bounding boxes' coordinates according to the real image width and height
        sub_r['center x'] = sub_r['center x'] / w
        sub_r['center y'] = sub_r['center y'] / h
        sub_r['width'] = sub_r['width'] / w
        sub_r['height'] = sub_r['height'] / h

        resulted_frame = sub_r.loc[:, ['CategoryID',
                                           'center x',
                                           'center y',
                                           'width',
                                           'height']].copy()

        # Checking if there is no any annotations for current image
        if resulted_frame.isnull().values.all():
            # Skipping this image
            continue

        # Saving resulted Pandas dataFrame into txt file
        path_to_save = path_dir + image_name + '.txt'
        if not os.path.exists(path_to_save):
          resulted_frame.to_csv(path_to_save, header=False, index=False, sep=' ')

        # Saving image in jpg format by OpenCV function
        path_to_save = path_dir + image_name + '.jpg'
        if not os.path.exists(path_to_save):
          cv2.imwrite(path_to_save, image_ppm)

In [ ]:
for file in path:
    # Checking if filename ends with '.ppm'
    if file.endswith('.ppm'):
      os.remove(file)


*   CREATING FILES TRAIN.TXT AND TEST.TXT




In [ ]:
import os

path_dir = '/content/darknet/FullIJCNN2013/'  # Update this to your directory path
files = os.listdir(path_dir)

p = []

for file in files:
    if file.endswith('.jpg'):
        path_to_save_into_txt_files = path_dir + file
        p.append(path_to_save_into_txt_files + '\n')

p_test = p[:int(len(p) * 0.15)]
p_train = p[int(len(p) * 0.15):]

with open('train.txt', 'w') as train_txt:
    train_txt.writelines(p_train)
    print("TRAIN DATA CREATED!")

with open('test.txt', 'w') as test_txt:
    test_txt.writelines(p_test)
    print("TEST DATA CREATED!")


TRAIN DATA CREATED!
TEST DATA CREATED!



*   CREATING TRAFFIC_SIGN_DATA.DATA AND CLASSES.NAMES




In [ ]:
NAME_COUNTER = 0
categories = ["prohibitory", "danger", "mandatory", "other"]
with open(path_dir + 'classes.names', 'w') as names:
    for cat in categories:
        names.write(cat + '\n')  # Copying all info from file txt to names
NUM_CATEGORY = len(categories)
NUM_CATEGORY

4

In [ ]:
data_path = "/content/darknet/cfg/"
with open(data_path + 'ts_data.data', 'w') as data:
    # Writing needed 5 lines
    # Number of classes
    # By using '\n' we move to the next line
    data.write('classes = ' + str(NUM_CATEGORY) + '\n')

    # Location of the train.txt file
    data.write('train = ' + path_dir + 'train.txt' + '\n')

    # Location of the test.txt file
    data.write('valid = ' + path_dir + 'test.txt' + '\n')

    # Location of the classes.names file
    data.write('names = ' + path_dir + 'classes.names' + '\n')

    # Location where to save weights
    data.write('backup = backup')

------------------------------------------------------------------------------

In [ ]:
%cd /content/darknet/FullIJCNN2013/
!pwd
!ls

/content/darknet/FullIJCNN2013
/content/darknet/FullIJCNN2013
00	   00095.txt  00194.txt  00292.txt  00412.txt  00521.txt  00681.txt  00810.txt
00000.jpg  00096.jpg  00195.jpg  00293.jpg  00413.jpg  00523.jpg  00682.jpg  00811.jpg
00000.txt  00096.txt  00195.txt  00293.txt  00413.txt  00523.txt  00682.txt  00811.txt
00001.jpg  00097.jpg  00196.jpg  00294.jpg  00414.jpg  00524.jpg  00683.jpg  00813.jpg
00001.txt  00097.txt  00196.txt  00294.txt  00414.txt  00524.txt  00683.txt  00813.txt
00002.jpg  00098.jpg  00197.jpg  00295.jpg  00415.jpg  00527.jpg  00685.jpg  00816.jpg
00002.txt  00098.txt  00197.txt  00295.txt  00415.txt  00527.txt  00685.txt  00816.txt
00003.jpg  00099.jpg  00198.jpg  00296.jpg  00416.jpg  00528.jpg  00686.jpg  00817.jpg
00003.txt  00099.txt  00198.txt  00296.txt  00416.txt  00528.txt  00686.txt  00817.txt
00004.jpg  00100.jpg  00199.jpg  00297.jpg  00417.jpg  00530.jpg  00688.jpg  00818.jpg
00004.txt  00100.txt  00199.txt  00297.txt  00417.txt  00530.txt  00688.t

In [ ]:

with open("/content/darknet/FullIJCNN2013/test.txt", "r") as file1:
    FileContent = file1.read()
    print(FileContent)

/content/darknet/FullIJCNN2013/00178.jpg
/content/darknet/FullIJCNN2013/00659.jpg
/content/darknet/FullIJCNN2013/00002.jpg
/content/darknet/FullIJCNN2013/00363.jpg
/content/darknet/FullIJCNN2013/00343.jpg
/content/darknet/FullIJCNN2013/00371.jpg
/content/darknet/FullIJCNN2013/00492.jpg
/content/darknet/FullIJCNN2013/00122.jpg
/content/darknet/FullIJCNN2013/00029.jpg
/content/darknet/FullIJCNN2013/00162.jpg
/content/darknet/FullIJCNN2013/00138.jpg
/content/darknet/FullIJCNN2013/00224.jpg
/content/darknet/FullIJCNN2013/00183.jpg
/content/darknet/FullIJCNN2013/00551.jpg
/content/darknet/FullIJCNN2013/00323.jpg
/content/darknet/FullIJCNN2013/00160.jpg
/content/darknet/FullIJCNN2013/00198.jpg
/content/darknet/FullIJCNN2013/00806.jpg
/content/darknet/FullIJCNN2013/00435.jpg
/content/darknet/FullIJCNN2013/00465.jpg
/content/darknet/FullIJCNN2013/00240.jpg
/content/darknet/FullIJCNN2013/00241.jpg
/content/darknet/FullIJCNN2013/00654.jpg
/content/darknet/FullIJCNN2013/00406.jpg
/content/darknet

**TRAINING**

In [ ]:
%cd ..
!ls

/content/darknet
3rdparty		darknet_images.py	    image_yolov3.sh	   README.md
backup			darknet.py		    image_yolov4.sh	   results
build			darknet_video.py	    include		   scripts
build.ps1		data			    json_mjpeg_streams.sh  src
cfg			docker-compose.yml	    LICENSE		   vcpkg.json
cmake			Dockerfile.cpu		    Makefile		   vcpkg.json.opencv23
CMakeLists.txt		Dockerfile.gpu		    net_cam_v3.sh	   video_yolov3.sh
darknet			FullIJCNN2013		    net_cam_v4.sh	   video_yolov4.sh
darknet53.conv.74	FullIJCNN2013classes.names  obj
DarknetConfig.cmake.in	FullIJCNN2013.zip	    package.xml


In [ ]:
# Start the training
!./darknet detector train cfg/ts_data.data cfg/yolov3_train.cfg darknet53.conv.74

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.781835), count: 8, class_loss = 0.228007, iou_loss = 0.265047, total_loss = 0.493054 
 total_bbox = 753115, rewritten_bbox = 0.147653 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.452983), count: 1, class_loss = 0.165227, iou_loss = 0.205672, total_loss = 0.370899 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.803977), count: 5, class_loss = 0.235953, iou_loss = 0.112681, total_loss = 0.348634 
 total_bbox = 753121, rewritten_bbox = 0.147652 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (m

In [ ]:
# Checking for all weights in order to define the weights with biggest mAP
!./darknet detector map cfg/ts_data.data cfg/yolov3_train.cfg backup/yolov3_train_1000.weights

 CUDA-version: 11080 (12000), cuDNN: 8.9.6, GPU count: 1  
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     27       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  27 0.252 BF
   1 conv     27       3 x 3/ 2    416 x 416 x  27 ->  208 x 208 x  27 0.568 BF
   2 conv     27       1 x 1/ 1    208 x 208 x  27 ->  208 x 208 x  27 0.063 BF
   3 conv     27       3 x 3/ 1    208 x 208 x  27 ->  208 x 208 x  27 0.568 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  27 0.001 BF
   5 conv     27       3 x 3/ 2    208 x 208 x  27 ->  104 x 104 x  27 0.142 BF
   6 conv     27       1 x 1/ 1    104 x 104 x  27 ->  104 x 104 x  27 0.016 BF
   7 conv     27       3 x 3/ 1    104 x 104 x  27 ->  104 x 104 x  27 0.142 BF
   8 Shortcut Layer: 5, 

**TESTING**

In [ ]:
!./darknet detector demo cfg/ts_data.data cfg/yolov3_ts_test.cfg weights/yolov3_train_1000.weights data/traffic-sign-to-test.mp4 -out_filename traffic-sign-to-test.avi -dont_show

 CUDA-version: 11080 (12000), cuDNN: 8.9.6, GPU count: 1  
 OpenCV version: 4.5.4
Demo
Couldn't open file: cfg/yolov3_ts_test.cfg
